In [1]:
import time
import random
import heapq
import math
from collections import deque

def listToInt(data):
    results = [ str(i) for i in data ]
    StrA = "".join(results)
    return int(StrA)

def getCollisionNum(board):
    sb = list(str(board))
    num = 0
    for col in range(len(sb)):
        for anotherCol in range(col+1, len(sb)):
            if sb[col] == sb[anotherCol]:
                num += 1 # collied in the same row
            elif abs(int(sb[col]) - int(sb[anotherCol])) == (anotherCol - col):
                num += 1 # collied diagonally
    return num


def sort_board(board):
    al = [1,2,3,4,5,6,7,8]
    other = []
    for i in range(0,9):
        for j in board :
            if i == j :
                try:
                    if al.index(i)!=-1 :
                        al.remove(i)
                except ValueError:
                    other.append(i)
    count = len(other)
    for i in range(0,len(other)) :
        val = other[i]
        idx = board.index(val)
        board[idx] = al[i]
    return board , count


class QueenStateInterface:
    def get_next_board(self):
        pass

    def get_successors(self):
        for i in range(0,8):
            for j in range(i+1,8) :
                target = self.board         
                tl = list(str(target))     
                x = tl[i]
                y = tl[j]
                tl[j] = x
                tl[i] = y
                yield self.get_next_board(listToInt(tl))
                
class QueenEncoding(QueenStateInterface):

    def get_next_board(self,target):
        tmp = QueensNState(self.board, self.inverse_board)
        tmp.board = target
        tmp.inverse_board = self.board
        return tmp
        

class Queen8State(QueenEncoding):
    base = [ 10**(7 - i) for i in range(8) ]

    def __init__(self, board, inverse_board):
        self.board = board
        self.inverse_board = inverse_board

    def __eq__(self, state):
        return self.board == state.board

    def __hash__(self):
        return self.board

    def from_data(data):
        
        board = listToInt(data)
        inverse_board = listToInt(data)
        return Queen8State(board, inverse_board)   
    
class Puzzle15State(QueenEncoding):
    base = [ 16**(15 - i) for i in range(16) ]

    def __init__(self, board, inverse_board):
        self.board = board
        self.inverse_board = inverse_board

    def __eq__(self, target):
        return self.board == target.board

    def __hash__(self):
        return self.board

    def from_data(data):
        board = sum(map(lambda a: a[0]*a[1], zip(data, Puzzle15State.base)))
        inverse_board = sum(map(lambda index: index*Puzzle15State.base[data[index]], range(9)))
        return Puzzle15State(board, inverse_board)
    
class QueensNState(QueenStateInterface):
    def __init__(self, board, inverse_board):

        self.board = board
        self.inverse_board = inverse_board

    def get_next_board(self,target):
        tmp = QueensNState(self.board, self.inverse_board)
        tmp.board = target
        tmp.inverse_board = self.board
        return tmp

    def __eq__(self, target):
        return self.board == target.board

    def __hash__(self):
        return hash(self.board)

    def from_data(board):
        inverse_board = board.copy()

        for i, val in enumerate(board):
            inverse_board[val] = i

        return PuzzleNState()
class SimpleQueenState:
    def __init__(self, state, g_value):
        self.state = state
        self.g_value = g_value    
        
class AStarPuzzleState:
    def __init__(self, state, g_value, h_value):
        self.state = state
        self.g_value = g_value
        self.h_value = h_value

    def __lt__(self, target):
        return (self.g_value + self.h_value) < (target.g_value + target.h_value)

class GreedyBFSQueensState:
    def __init__(self, state, g_value, h_value):
        self.state = state
        self.g_value = g_value
        self.h_value = h_value

    def __lt__(self, target):
        return self.h_value < target.h_value

class RBFSPuzzleState:
    def __init__(self, state, g_value, f_value):
        self.state = state
        self.g_value = g_value
        self.f_value = f_value

    def __lt__(self, target):
        return self.f_value < target.f_value


class SolutionPack:
    class VisitGuard:
        def __init__(self, state, visited_map):
            self.state = state
            self.visited_map = visited_map
        def __enter__(self):
            self.visited_map[self.state] = True
        def __exit__(self, exc_type, exc_value, traceback):
            self.visited_map[self.state] = False

    class NewStateChecker:
        def __init__(self, state_set):
            self.state_set = state_set
            self.is_new_insert = False

        def attempt_insert(self, state):
            if (state not in self.state_set):
                self.is_new_insert = True
                self.state_set.add(state)

        def __enter__(self):
            self.is_new_insert = False

        def __exit__(self, exc_type, exc_value, traceback):
            pass    
        
    def iterative_deepening_search(self, src):

        state_set = set()
        visited_map = {}
        checker = SolutionPack.NewStateChecker(state_set)
        checker.attempt_insert(src)
        
        def depth_limit_search(current, depth_quota):
            if depth_quota == 0:
                if getCollisionNum(current.state.board) == 0:
                    return (current.g_value, len(state_set) + 1)
                else:
                    return None

            visited_map[current.state] = True

            for successor in current.state.get_successors():
                if (successor not in visited_map or not visited_map[successor]):
                    checker.attempt_insert(successor)
                    result = depth_limit_search(SimpleQueenState(successor, current.g_value + 1), depth_quota - 1)

                    if result:
                        return result

            visited_map[current.state] = False

            return None
        
        for depth in map(sum, enumerate(iter(int, 1), start=1)):
            with checker:
                result = depth_limit_search(SimpleQueenState(src, 0), depth)

                if result:
                    return result
                elif not checker.is_new_insert:
                    return None

    def uniform_cost_search(self, src):

        visited_set = set()
        queue = deque()
        
        visited_set.add(src)
        queue.append(SimpleQueenState(src, 0))

        while queue:
            current = queue.popleft()
            for successor in current.state.get_successors():
                if successor not in visited_set:
                    visited_set.add(successor)
                    collisionNum = getCollisionNum(successor.board)
                    if collisionNum == 0:
                        return (current.g_value + 1, len(visited_set))
                    else:
                        queue.append(SimpleQueenState(successor, current.g_value + 1))
    def greedy_best_first_search(self,src):
        visited_set = set()
        queue = []
    
        heapq.heappush(queue, GreedyBFSQueensState(src, 0, getCollisionNum(src.board)))
        while queue:
            current = heapq.heappop(queue)
            for successor in current.state.get_successors():
                if successor not in visited_set:
                    visited_set.add(successor)
                    collisionNum = getCollisionNum(successor.board)
                    if collisionNum == 0:
                        return (current.g_value + 1, len(visited_set))
                    else:
                        heapq.heappush(queue, GreedyBFSQueensState(successor, current.g_value + 1, collisionNum))
    def astar_search(self, src):

        visited_set = set()
        queue = []
        heapq.heappush(queue, AStarPuzzleState(src, 0, getCollisionNum(src.board)))

        while queue:
            current = heapq.heappop(queue)
            for successor in current.state.get_successors():
                if successor not in visited_set:
                    visited_set.add(successor)
                    collisionNum = getCollisionNum(successor.board)
                    if collisionNum == 0:
                        return (current.g_value + 1, len(visited_set))
                    else:
                        heapq.heappush(queue, AStarPuzzleState(successor, current.g_value + 1,collisionNum))
    
    def recursive_best_first_search(self, src):

        visited_map = {}

        def rbfs_search(current, f_limit):
            if getCollisionNum(current.state.board) == 0:
                return (current.g_value, len(visited_map))

            successors = [ 
                RBFSPuzzleState(
                    successor, 
                    current.g_value + 1, 
                    max(current.g_value + getCollisionNum(successor.board) + 1, current.f_value)
                ) for successor in current.state.get_successors()
            ]

            with SolutionPack.VisitGuard(current.state, visited_map) as guard:
                if not successors:
                    current.f_value = 1 << 31
                    return None
                elif len(successors) == 1:
                    result = rbfs_search(successors[0], f_limit)
                    current.f_value = successors[0].f_value
                    return result
                else:
                    heapq.heapify(successors)
                    best = successors[0]
                    while True:
                        best = heapq.heappop(successors)
                        if best.f_value > f_limit:
                            break;

                        result = rbfs_search(best, min(successors[0].f_value, f_limit))

                        if result:
                            return result
                        else:
                            heapq.heappush(successors, best)
                    current.f_value = best.f_value
                    return None
        return rbfs_search(RBFSPuzzleState(src, 0, getCollisionNum(src.board)), 1 << 31)      

In [2]:
# 八皇后
queen8_src = [1,3,5,4,3,2,1,7]

In [3]:
src,count = sort_board(queen8_src)
sol_pack = SolutionPack()

src,count



([6, 8, 5, 4, 3, 2, 1, 7], 2)

In [6]:
src_state = Queen8State.from_data(src)
cost,stats = sol_pack.iterative_deepening_search(src_state)
print("iterative_deepening_search 步數:",cost+count,"||狀態數:",stats+count)
cost,stats = sol_pack.uniform_cost_search(src_state)
print("uniform_cost_search  步數:",cost+count,"||狀態數:",stats+count)
cost,stats = sol_pack.greedy_best_first_search(src_state)
print("greedy_best_first_search  步數:",cost+count,"||狀態數:",stats+count)
cost,stats = sol_pack.astar_search(src_state)
print("astar_search  步數:",cost+count,"||狀態數:",stats+count)
cost,stats = sol_pack.recursive_best_first_search(src_state)
print("recursive_best_first_search  步數:",cost+count,"||狀態數:",stats+count)

iterative_deepening_search 步數: 12 ||狀態數: 9941
uniform_cost_search  步數: 12 ||狀態數: 9940
greedy_best_first_search  步數: 14 ||狀態數: 171
astar_search  步數: 12 ||狀態數: 149
recursive_best_first_search  步數: 34 ||狀態數: 19


In [5]:
# 一皇后
queen8_src = [1,0,0,0,0,0,0,0]

src,count = sort_board(queen8_src)
sol_pack = SolutionPack()
src_state = Queen8State.from_data(src)
cost,stats = sol_pack.iterative_deepening_search(src_state)
print("iterative_deepening_search 步數:",cost+count,"||狀態數:",stats+count)
cost,stats = sol_pack.uniform_cost_search(src_state)
print("uniform_cost_search  步數:",cost+count,"||狀態數:",stats+count)
cost,stats = sol_pack.greedy_best_first_search(src_state)
print("greedy_best_first_search  步數:",cost+count,"||狀態數:",stats+count)
cost,stats = sol_pack.astar_search(src_state)
print("astar_search  步數:",cost+count,"||狀態數:",stats+count)
cost,stats = sol_pack.recursive_best_first_search(src_state)
print("recursive_best_first_search  步數:",cost+count,"||狀態數:",stats+count)

iterative_deepening_search 步數: 12 ||狀態數: 9941
uniform_cost_search  步數: 12 ||狀態數: 9940
greedy_best_first_search  步數: 14 ||狀態數: 171
astar_search  步數: 12 ||狀態數: 149
recursive_best_first_search  步數: 34 ||狀態數: 19
